In [ ]:
import pandas as pd
import numpy as np
from pandas.tseries.offsets import Day
import matplotlib.pyplot as plt
import pgeocode
import plotly.express as px
import seaborn as sns
from calendar import day_abbr, month_abbr, mdays
import holidays

In [ ]:
%matplotlib inline

# Load Solar home electricity datasets:

* In our project, we'll use two data sets of Solar home electricity for two years, (2011-2012) and (2012-2013)

In [ ]:
df_1 = pd.read_csv('2011-2012 Solar home electricity data v2.csv', skiprows=1,
                    parse_dates=['date'], dayfirst=True,
                    na_filter=False, dtype={'Row Quality': str})
df_1.tail()

In [ ]:
df_2 = pd.read_csv('2012-2013 Solar home electricity data v2.csv', skiprows=1,
                    parse_dates=['date'], dayfirst=True,
                    na_filter=False, dtype={'Row Quality': str})
df_2.tail()

### Check the missing values:

First, we'll drop all Non-Actual values using the Row Quality feature as it has two values:
* Row Quality is (Blank) which means every half-hour value in the row is the actual electricity recorded by the meter in the half-hour.
* Row Quality is NA that means Non-Actual where some or all of the half-hour values in the row are estimates or substitutes of the electricity consumed or generated.

In [ ]:
print(df_1['Row Quality'].value_counts())
df_1.drop(df_1[df_1['Row Quality'] == 'NA'].index, inplace=True)

In [ ]:
df_1['Row Quality'].value_counts()

In [ ]:
print(df_2['Row Quality'].value_counts())
df_2.drop(df_2[df_2['Row Quality'] == 'NA'].index, inplace=True)

In [ ]:
df_2['Row Quality'].value_counts()

Second, we'll check if the data frames has any other missing (null) values.

In [ ]:
df_1.isnull().sum()

In [ ]:
df_2.isnull().sum()

* Note, we'll keep all Zeros values in General Consumption since it's ilogical for general power consumption to be zero, which indicates Abnormal behaviour in the data. 

In [ ]:
df_1[df_1['Consumption Category']=='GC'].eq(0).sum()
df_2[df_2['Consumption Category']=='GC'].eq(0).sum()

# Explor and prepare the datasets:

* First, we'll explore the postcodes of the costumers.
* Second, we'll create a new dataset, which contains about 10 customers based on postcode.

In [ ]:
print(df_1['Postcode'].unique())
print(len(df_1['Postcode'].unique()))

In [ ]:
print(df_2['Postcode'].unique())
print(len(df_2['Postcode'].unique()))

In [ ]:
df_1['Customer'].unique()

In [ ]:
df_2['Customer'].unique()

* We have 100 different postcodes and 300 customers, for our project, we'll choose 10 customers who have the same postcode and located in the same area

In [ ]:
# Print the top value of Postcode for both datasets

print("Postcode Value_counts")
print(df_1['Postcode'].value_counts().nlargest(1))
print(df_2['Postcode'].value_counts().nlargest(1))

In [ ]:
# Choose the top customers at this postcode for both datasets

s1 = df_1.Customer[df_1['Postcode']==2259].value_counts().nlargest(13).to_frame().index
s2 = df_2.Customer[df_2['Postcode']==2259].value_counts().nlargest(13).to_frame().index
print(s1,'\n',s2)

In [ ]:
# Select the 10 customers which intersection in both datasets

selected_customer_id  = np.sort(s1[s1.isin(s2)])
print(selected_customer_id)

In [ ]:
# Filter the first dataset using selected customers Ids

selected_df_1 = df_1[df_1.Customer.isin(selected_customer_id)]
selected_df_1.head()

In [ ]:
# Filter the second  dataset using selected customers IDs

selected_df_2 = df_2[df_2.Customer.isin(selected_customer_id)]
selected_df_2.head()

In [ ]:
# Check that both datasets have the same customers

print(selected_df_1.Customer.unique())
print(selected_df_2.Customer.unique())

In [ ]:
# Check that both datasets have the same postcode

print(selected_df_2.Postcode.unique())
print(selected_df_2.Postcode.unique())

##### Plot the postcodes:

* The postcodes in our datasets are between 2008 and 2330, which relates to New South Wales.
* The plot indicates that the postcode we chose contained the biggest number of customers.

In [ ]:
Nomin = pgeocode.Nominatim('au')

In [ ]:
postcode_1 = df_1['Postcode'].value_counts().to_frame().reset_index()
postcode_1.columns = ['Postcode','Value']
postcode_2 = df_2['Postcode'].value_counts().to_frame().reset_index()
postcode_2.columns = ['Postcode','Value']

In [ ]:
postcode = pd.DataFrame()
postcode['Postcode'] = postcode_1.Postcode[postcode_1.Postcode.isin(postcode_2.Postcode)]
postcode['Value'] = postcode_1.Value[postcode_1.Postcode.isin(postcode_2.Postcode)] + postcode_2.Value[postcode_1.Postcode.isin(postcode_2.Postcode)]

In [ ]:
postcode["Latitude"] =postcode["Postcode"].apply(lambda x: Nomin.query_postal_code(x)[9])
postcode["Longitude"] = postcode["Postcode"].apply(lambda x: Nomin.query_postal_code(x)[10])
postcode.head()

In [ ]:
conda install -c plotly plotly-orca

In [ ]:
fig = px.scatter_mapbox(postcode, lat="Latitude", lon="Longitude", zoom=7, height=400,
                       color="Postcode", size= "Value",color_continuous_scale=px.colors.cyclical.IceFire, size_max=15,
                        center=dict(lat=-33.1763, lon=151.461652),
                       mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()
#fig.write_image("fig2.pdf") 

### Reshape and merge the datasets 

* The reference of this part of code is from: https://github.com/pierre-haessig/ausgrid-solar-data

In [ ]:
# Stack the time, to get proper timestamp combining day and hour

df_1_min, df_1_max = selected_df_1.date.min(), selected_df_1.date.max()
df_2_min, df_2_max = selected_df_2.date.min(), selected_df_2.date.max()
print(df_1_min, df_1_max)
print(df_2_min, df_2_max )

In [ ]:
# Create the index, with "left" convention (start of the 30min interval), to make slicing for a given day easier

Date_index_1 = pd.date_range(df_1_min, df_1_max + Day(1), freq='30T', closed='left')
Date_index_2 = pd.date_range(df_2_min, df_2_max + Day(1), freq='30T', closed='left')

#### Consumption categories

The datasets have three Categories:

* GC = General Consumption for electricity supplied all the time (primary tariff, either inclining block or time of use rates), excluding solar generation and controlled load supply
* CL = Controlled Load Consumption (Off peak 1 or 2 tariffs)
* GG = Gross Generation for electricity generated by the solar system with a gross metering configuration, measured separately to household loads

In our project, we interested to the power consumption, not power generation, due to this reason, we'll use General power consumption category.

In [ ]:
print(selected_df_1['Consumption Category'].unique())
print(selected_df_2['Consumption Category'].unique())


Create a new data frame using an empty MultiIndex (Customers/Features).

In [ ]:
customers = selected_customer_id
customers

In [ ]:
# here are all the features that we will extracted later in addition to GC (General consumption).

Features = ['hourofday','minuteofhour','dayofweek','dayofmonth','monthofyear',
            'year','GC','Anomaly']


In [ ]:
columns = pd.MultiIndex.from_product(
    (customers, Features), names=['Customer', 'Features'])
columns

In [ ]:
empty_cols = pd.MultiIndex(
    levels=[customers, Features],
    codes=[[],[]],
    names=['Customer', 'Features'])

In [ ]:
empty_cols

In [ ]:
Data_set_1 = pd.DataFrame(index=Date_index_1, columns=empty_cols)
Data_set_1

In [ ]:
Data_set_2 = pd.DataFrame(index=Date_index_2, columns=empty_cols)
Data_set_2

### Features Extraction:

In [ ]:
for c in customers:
    d_c = selected_df_1[selected_df_1.Customer == c] 
    print(c, end=', ')
    d_c_ch = d_c[d_c['Consumption Category'] == 'GC']
    ts = d_c_ch.iloc[:,5:-1].values.ravel()
    Data_set_1[c, 'GC'] = ts 
    Data_set_1[c,'hourofday']=Data_set_1[c].index.hour
    Data_set_1[c,'minuteofhour']=Data_set_1[c].index.minute
    Data_set_1[c,'dayofweek']=Data_set_1[c].index.dayofweek
    Data_set_1[c,'dayofmonth']=Data_set_1[c].index.day
    Data_set_1[c,'monthofyear']=Data_set_1[c].index.month
    Data_set_1[c,'year']=Data_set_1[c].index.year


In [ ]:
Data_set_1.head()

In [ ]:
for c in customers:
    d_c = selected_df_2[selected_df_2.Customer == c] 
    print(c, end=', ')
    d_c_ch = d_c[d_c['Consumption Category'] == 'GC']
    ts = d_c_ch.iloc[:,5:-1].values.ravel()
    Data_set_2[c, 'GC'] = ts
    Data_set_2[c,'hourofday']=Data_set_2[c].index.hour
    Data_set_2[c,'minuteofhour']=Data_set_2[c].index.minute
    Data_set_2[c,'dayofweek']=Data_set_2[c].index.dayofweek
    Data_set_2[c,'dayofmonth']=Data_set_2[c].index.day
    Data_set_2[c,'monthofyear']=Data_set_2[c].index.month
    Data_set_2[c,'year']=Data_set_2[c].index.year


In [ ]:
Data_set_2.head()

In [ ]:
'''we will not have any misining records, since we removed Non-Actual values and
filtered the datasets with selected customers-id'''

All_data= pd.concat([Data_set_1, Data_set_2], axis = 0, levels = 0)
All_data

In [ ]:
All_data.describe()

# ========================

### Some Data exploration and plots:

In [ ]:
power_c = All_data.xs('GC', level='Features', axis=1)

In [ ]:
power_c[selected_customer_id].resample('1Y').sum().transpose().plot(kind="bar",figsize=(20,13),colormap="winter")

plt.legend(['2011','2012','2013'],loc='upper left',title='Years',fontsize=20)
#plt.title('Total power consumption over two years for the ten selected customers \n',fontsize="x-large")
plt.xlabel('\n Customers-IDs',fontsize=25)
plt.ylabel('Power consumption KW/h',fontsize=25)
plt.rc('xtick', labelsize=25) 
plt.rc('ytick', labelsize=25) 
plt.savefig('fig3.pdf')

In [ ]:
power_c[selected_customer_id].sum(axis=1).resample('3M').sum().plot(kind="bar",figsize=(20,13),width=0.2,edgecolor='g',color='y')
locs, labels = plt.xticks()
plt.xticks(locs, ['Winter\n2011', 'Spring\n2011', 'Summer\n2012','Autumn\n2012','Winter\n2012', 'Spring\n2012',
         'Summer\n2013','Autumn\n2013','Winter\n2013'],rotation=0) 
plt.xlabel('\n Seasons of the year',fontsize=25)
plt.ylabel('Power consumption KW/h',fontsize=25)
#plt.title('Total Seasonal power consumption over two years for the ten selected customers IDs \n',fontsize="x-large")
plt.rc('xtick', labelsize=25) 
plt.rc('ytick', labelsize=25) 
plt.savefig('fig4.pdf')

In [ ]:
power_c[selected_customer_id].resample('3M').sum().plot(kind="bar",figsize=(15,7))
locs, labels = plt.xticks()
plt.xticks(locs, ['Winter/2011', 'Spring/2011', 'Summer/2012','Autumn/2012','Winter/2012', 'Spring/2012',
         'Summer/2013','Autumn/2013','Winter/2013'],rotation=0) 
plt.legend(loc='upper right',ncol=5, title="Customers ID")
plt.xlabel('\n Seasons of the year',fontsize=13)
plt.ylabel('Power consumption KW/h',fontsize=13)
plt.title('Total Seasonal power consumption over two years for the ten selected customers IDs \n',fontsize=15)

#plt.savefig('Total Seasonal power consumption.png', dpi=150)

In [ ]:
power_data= power_c[selected_customer_id].sum(axis=1).groupby([power_c.index.dayofweek,power_c.index.month]).sum().unstack()
power_data.plot(style='-D',figsize=(20,12))
plt.legend(loc='best',ncol=6,title='Months',labels=month_abbr[1:13],fontsize=20)
plt.xlabel('Day of the week', fontsize=25)
plt.xticks([0,1,2,3,4,5,6],['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'])
plt.ylabel('Power consumption KW/h', fontsize=25)
#plt.title('Total energy consumption by month and day for all customers',fontsize="x-large")
plt.rc('xtick', labelsize=25) 
plt.rc('ytick', labelsize=25) 
plt.savefig('fig5.pdf')
plt.show()


In [ ]:
power_data= power_c[selected_customer_id].sum(axis=1).groupby([power_c.index.dayofweek,power_c.index.hour]).sum().unstack()
f, ax = plt.subplots(figsize=(16,8))
sns.heatmap(power_data, ax = ax, cmap='Blues')
cbax = f.axes[1]
[l.set_fontsize(13) for l in cbax.yaxis.get_ticklabels()]
cbax.set_ylabel('Power consumption kw/h', fontsize=25)
#ax.set_title('Power consumption per day of the week and hour of the day for all customers', fontsize="x-large")
[l.set_fontsize(11) for l in ax.xaxis.get_ticklabels()]
[l.set_fontsize(11) for l in ax.yaxis.get_ticklabels()]
ax.set_xlabel('hour of the day', fontsize=25)
ax.set_ylabel('day of the week', fontsize=25)
ax.set_yticklabels(day_abbr[0:7])
plt.rc('xtick', labelsize=25) 
plt.rc('ytick', labelsize=25) 
plt.savefig('fig6.pdf')
plt.show()

# FB prophet:

* The refrence of this part of code :
   - https://facebook.github.io/prophet/docs/installation.html#python
   - https://towardsdatascience.com/anomaly-detection-time-series-4c661f6f165f
   - https://medium.com/seismic-data-science/anomaly-detection-using-prophet-a5dcea2c5473
   - https://www.kaggle.com/vinayjaju/anomaly-detection-using-facebook-s-prophet
   - https://nbviewer.jupyter.org/github/nicolasfauchereau/Auckland_Cycling/blob/master/notebooks/Auckland_cycling_and_weather.ipynb

In [ ]:
from matplotlib.dates import MonthLocator, num2date
from matplotlib.ticker import FuncFormatter
from fbprophet import Prophet
import itertools

In [ ]:
import warnings
# prophet preformance
from fbprophet.diagnostics import cross_validation
from fbprophet.diagnostics import performance_metrics
from fbprophet.plot import plot_cross_validation_metric
# "high resolution"
%config InlineBackend.figure_format = 'retina'
warnings.filterwarnings('ignore')

In [ ]:
import os
import altair as alt
alt.renderers.enable('default')
alt.data_transformers.disable_max_rows()
#alt.renderers.enable('notebook')
print(os.listdir("/Users/eserrari/Desktop/All folders/Thesis submission/SourceCode_MSThesis/part-1"))
from IPython.display import HTML

#HTML("This code block contains import statements and setup.")

### Grid Search hyperparameter tuning:

For tuning hyperparameters of the model we used cross-validation, we defined a param_grid of all the parameters and values we want to loop through, and then calculated the performance matrix, and get the best parameter combination in terms of RMSE, averaged over a 180-day horizon. we compare around 192 models with parallelization over cutoffs.

In [ ]:
def tuning_func(new_data):
    param_grid = {  
        'changepoint_range': [0.8, 0.9],
        'changepoint_prior_scale': [0.001, 0.1, 0.5,1.0],
        'seasonality_prior_scale': [0.01, 0.1, 1.0, 10.0],
        'holidays_prior_scale': [0.01, 0.1, 1.0],
        'seasonality_mode': ['additive', 'multiplicative'],
    }

    # Generate all combinations of parameters
    all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
    rmses = []  # Store the RMSEs for each params here

    # Use cross validation to evaluate all parameters
    cutoffs = pd.to_datetime(['2012-12-01'])
    for params in all_params:
        m = Prophet(**params).fit(new_data)  # Fit model with given params
        df_cv = cross_validation(m, cutoffs=cutoffs, horizon='180 days', parallel="processes")
        df_p = performance_metrics(df_cv, rolling_window=1)
        rmses.append(df_p['rmse'].values[0])

    # Find the best parameters
    tuning_results = pd.DataFrame(all_params)
    tuning_results['rmse'] = rmses
    best_params = all_params[np.argmin(rmses)]

    # plot the RMSE for the forecast.
    fig = plot_cross_validation_metric(df_cv, metric='rmse')
    plt.show()
    return best_params,df_p

#### Note:
    * The next part of the code needs around 40 hours to be finished. 

In [ ]:
best_param = dict()
cv_metrics = dict()
for i in selected_customer_id:
    temp_data = power_c[i].reset_index()
    temp_data.columns = ['ds','y']
    best_param[i],cv_metrics[i] = tuning_func(temp_data)

In [ ]:
# Save Best parameters to a file
np.save('best_params.npy', best_param) 

In [ ]:
# Load the Best parameters file to avoid run the hyper-tuning code part
best_param = np.load('best_params.npy',allow_pickle='TRUE').item()

In [ ]:
for k, v in best_param.items():
    print('Best parameters for Customer ',k)
    print(v,'\n')

In [ ]:
for k, v in cv_metrics.items():
    print('The CV scors for Customer ',k)
    print(v,'\n')

### Prophet functions:

In [ ]:
def is_weekends(ds):
    date = pd.to_datetime(ds)
    return (date.weekday() == 5 or date.weekday() == 6 )


def fit_predict_model(dataframe,best_params):
    dataframe['weekends'] = dataframe['ds'].apply(is_weekends)
    dataframe['week_days'] = ~dataframe['ds'].apply(is_weekends)
    
    m = Prophet(daily_seasonality = True, 
                weekly_seasonality = False,
                yearly_seasonality = True, 
                interval_width = 0.99,
                changepoint_range = best_params['changepoint_range'],
                changepoint_prior_scale = best_params['changepoint_prior_scale'], 
                seasonality_prior_scale = best_params['seasonality_prior_scale'],
                holidays_prior_scale = best_params['holidays_prior_scale'],
                seasonality_mode = best_params['seasonality_mode'])
    
    m.add_country_holidays(country_name='AU')
    m.add_seasonality(name='week_ends', period=7, fourier_order=3, condition_name='weekends')
    m.add_seasonality(name='week_days', period=7, fourier_order=3, condition_name='week_days')

    m = m.fit(dataframe)
    forecast = m.predict(dataframe)
    forecast['fact'] = dataframe['y'].reset_index(drop = True)
    print('Displaying Prophet components plot')
    
    plt.rc('xtick', labelsize=15) 
    plt.rc('ytick', labelsize=15) 
    fig1 = m.plot(forecast,figsize=(16, 12))
    plt.xlabel('ds',fontsize=20)
    plt.ylabel('y',fontsize=20)
    #plt.savefig('fig7.pdf')
    
    plt.rc('xtick', labelsize=11) 
    plt.rc('ytick', labelsize=11) 
    fig2 = m.plot_components(forecast)
    #plt.savefig('fig8.pdf')
    plt.show()
    return forecast


In [ ]:
def detect_anomalies(forecast):
    forecasted = forecast[['ds','trend', 'yhat', 'yhat_lower', 'yhat_upper', 'fact']].copy()
    forecasted['anomaly'] = 0

    # identify any value lies outside the upper bound of the confidence interval as outliers 
    forecasted.loc[forecasted['fact'] > forecasted['yhat_upper'], 'anomaly'] = 1
    # identify any value lies outside the upper bound of the confidence interval as outliers 
    forecasted.loc[forecasted['fact'] < forecasted['yhat_lower'], 'anomaly'] = 1
    # identify all zero values as outliers 
    forecasted.loc[forecasted['fact'] == 0, 'anomaly'] = 1
    '''We can add this line to identify any negative values as outliers,
     In our dataset we don’t have any negative values '''
    #forecasted.loc[forecasted['fact'] < 0, 'anomaly'] = 1
    
    #Calculate anomaly importances, to use it only with anomalies plotting 
    forecasted['importance'] = 0
    Interval_range = forecasted['yhat_upper'] - forecasted['yhat_lower']
    forecasted.loc[forecasted['fact'] > forecasted['yhat_upper'], 'importance'] = \
        (forecasted['fact'] - forecasted['yhat_upper'])/Interval_range
    forecasted.loc[forecasted['fact'] < forecasted['yhat_lower'], 'importance'] = \
        (forecasted['yhat_lower'] - forecasted['fact'])/Interval_range
    # assigned the same imortance score for the GC with zero values
    forecasted.loc[(forecasted['anomaly'] ==1) & (forecasted['fact'] == 0), 'importance'] = 0.25
    
    # assigned the same imortance score for the GC with negative values, this part can be added if we have negative values
    #forecasted.loc[(forecasted['anomaly'] ==1) & (forecasted['fact'] < 0), 'importance'] = 0.25

    return forecasted

In [ ]:
def plot_anomalies(forecasted):  
    interval = alt.Chart(forecasted).mark_area( color = '#F7E98E').encode(
    x=alt.X('ds:T',  title ='Date_Time'),
    y='yhat_upper',
    y2='yhat_lower',
    tooltip=['hoursminutes(ds)', 'fact', 'yhat_lower', 'yhat_upper']
    ).interactive().properties(
        title='Anomaly detection over six months'
    )
    
    fact = alt.Chart(forecasted[forecasted.anomaly==0]).mark_circle(size=20, opacity=0.7, color = 'Black').encode(
        x='ds:T',
        y=alt.Y('fact'),    #, title='Globel power'
        tooltip=['hoursminutes(ds)', 'fact', 'yhat_lower', 'yhat_upper']
    ).interactive()

    anomalies = alt.Chart(forecasted[forecasted.anomaly!=0]).mark_circle(size=5, color = '#73b27d').encode(
        x='ds:T',
        y=alt.Y('fact', title='Power consumption KW/h'),    
        tooltip=['hoursminutes(ds)', 'fact', 'yhat_lower', 'yhat_upper'],
        size = alt.Size( 'importance', legend=None)
    ).interactive()

    render = (alt.layer(interval, fact, anomalies)\
              .properties(width=870, height=450)\
              .configure_title(fontSize=20).configure_axis(
                labelFontSize=18,titleFontSize=20
                )).display() 
    return render

### Plot functions:

In [ ]:
def plot_1(power_data,i):
    power_data = power_data[['GC','dayofweek','monthofyear']]
    power_data = power_data.groupby(['dayofweek','monthofyear']).sum().unstack()
    power_data.columns = power_data.columns.droplevel(0)

    power_data.plot(style='-D',figsize=(20,8))
    plt.legend(loc='best',ncol=6,title='Months',labels=month_abbr[1:13])
    plt.xlabel('Day of the week', fontsize=13)
    plt.xticks([0,1,2,3,4,5,6],['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'])
    plt.ylabel('Power consumption KW/h', fontsize=13)
    plt.title('Total energy consumption by month and day for customer-ID: %i' %i,fontsize=15)
    plt.show()
    #plt.savefig(Total energy consumption.{jpeg}', dpi=200)
    return

In [ ]:
def plot_2(power_data,i):
    power_data = power_data[['GC','dayofweek','hourofday']]
    power_data = power_data.groupby(['dayofweek','hourofday']).sum().unstack()
    power_data.columns = power_data.columns.droplevel(0)
    
    f, ax = plt.subplots(figsize=(12,6))
    sns.heatmap(power_data, ax = ax, cmap='Blues')
    cbax = f.axes[1]
    [l.set_fontsize(13) for l in cbax.yaxis.get_ticklabels()]
    cbax.set_ylabel('Power consumption kw/h', fontsize=13)
    ax.set_title('Power consumption per day of the week and hour of the day for customer-ID: %i' %i, fontsize=15)
    [l.set_fontsize(11) for l in ax.xaxis.get_ticklabels()]
    [l.set_fontsize(11) for l in ax.yaxis.get_ticklabels()]
    ax.set_xlabel('hour of the day', fontsize=13)
    ax.set_ylabel('day of the week', fontsize=13)
    ax.set_yticklabels(day_abbr[0:7])
    plt.show()
 
    return

In [ ]:
def plot_3(power_data,i):
    f, ax = plt.subplots(figsize=(15,7))
    
    ax.plot(power_data.GC[~power_data.index.dayofweek.isin([5,6])].resample('1W').mean(), color='g', label='week days', lw=2)
    ax.plot(power_data.GC[power_data.index.dayofweek.isin([5,6])].resample('1W').mean(), color='b', label='week-ends', ls='--', lw=2)
    ax.grid(ls=':', color='0.8')
    ax.set_xlabel('Month of the year', fontsize=13)
    ax.set_ylabel('Power Consumption KW/h', fontsize=13)
    ax.legend(['week days','week-ends'] , fontsize=13)
    ax.set_title('Average weekly power consumption during week days versus week-ends for customer-ID: %i' %i, fontsize=15)
    plt.show()
    return

In [ ]:
def plot_4(power_data,i,j):
    f, ax = plt.subplots(figsize=(20,10))
    ax.plot(power_data.GC, lw=2)
    ax.grid(ls=':', color='0.8')
    ax.set_xlabel('Date', fontsize=25)
    ax.set_ylabel('Power Consumption KW/h', fontsize=25)
    ax.set_title('Half-hour power consumption of Community Coustomer-ID: %i'%i+' over %i months' %j, fontsize=25)
    # This part to be used with thesis discussion 
    #ax.fill_between(power_data.index, power_data['lower'], power_data['upper'], alpha=0.2,color='orange')
    #scatter = ax.scatter(power_data.GC[power_data['Anomaly']==1].index,power_data.GC[power_data['Anomaly']==1], color='red',label='Anomaly')
    plt.rc('xtick', labelsize=25) 
    plt.rc('ytick', labelsize=25)
   # plt.savefig('fig14.pdf')
    plt.show()
    return

In [ ]:
def plot_5(power_data,i,j):
    f, ax = plt.subplots(figsize=(20,10))
    ax.plot(power_data.GC[power_data['Anomaly']==0], color='g', lw=1,label='power consumption')
    scatter = ax.scatter(power_data.GC[power_data['Anomaly']==1].index,power_data.GC[power_data['Anomaly']==1], color='red',label='Anomaly')
    ax.legend(loc="upper right",labelcolor='k', fontsize=18)
    ax.grid(ls=':', color='0.8')
    ax.set_xlabel('Date', fontsize=25)
    ax.set_ylabel('Power Consumption KW/h', fontsize=25)
    ax.set_title('Anomalies of half-hour power consumption over %i' %j + ' months of Community Customer-ID: %i' %i, fontsize=25)
    # This part to be used with thesis discussion 
    #ax.fill_between(power_data.index, power_data['lower'], power_data['upper'], alpha=0.2,color='orange')
    plt.rc('xtick', labelsize=25) 
    plt.rc('ytick', labelsize=25) 
    #plt.savefig('fig15.pdf')
    plt.show()
    return

### Identify the Anomalies for each Customer:

In [ ]:
selected_customer_id

#### First customer :

In [ ]:
pred = dict()
i = selected_customer_id[0]
print("\n\n Identifying the Anomalies for Customer ==> ",i)
    
print("\n\n Step-1: Fit FB-prophet model with best parameters")
temp_data= All_data.xs('GC', level='Features', axis=1)[i].reset_index()
temp_data.columns=['ds','y']
print("\n Plot the FB-prophet model for Customer ==>",i)
pred[i] = fit_predict_model(temp_data,best_param[i])

print("\n\n Step-2 : Detect the anamalies for Customer ==>",i)
pred[i] = detect_anomalies(pred[i])
print("\n Plot the anamalies for Customer ==>",i)
plot_anomalies(pred[i])

print("Customer",i," values counts: \n",pred[i]['anomaly'].value_counts())
print("\n\n Step-3: Assign the Anomalies to the Customer",i)
All_data[i,'Anomaly'] = pred[i]['anomaly'][All_data[i].index==pred[i]['ds']].values.ravel()
# This part to be used with thesis discussion 
#All_data[i,'lower'] = pred[i]['yhat_lower'][All_data[i].index==pred[i]['ds']].values.ravel()
#All_data[i,'upper'] = pred[i]['yhat_upper'][All_data[i].index==pred[i]['ds']].values.ravel()
print("\n Customer ",i,"Is Done!" )
print("==============================================================================")

In [ ]:
#plot_anomalies(pred[i].loc[(pred[i].ds > '2012-07-01')])
plot_anomalies(pred[i].loc[(pred[i].ds > '2013-01-01')])
#pred[i].loc[(pred[i].ds > '2012-01-01') & (pred[i].ds  < '2012-03-30')]

In [ ]:
plot_1(All_data[i],i)

In [ ]:
plot_2(All_data[i],i)

In [ ]:
plot_3(All_data[i],i)

In [ ]:
plot_4(All_data[i]['2012-01-01':'2012-03-31'],i,3)

In [ ]:
plot_5(All_data[i]['2012-01-01':'2012-03-30'],i,3)

# ===============================================

#### Second customer:

In [ ]:

i = selected_customer_id[1]
print("\n\n Identifying the Anomalies for Customer ==> ",i)
    
print("\n\n Step-1: Fit FB-prophet model with best parameters")
temp_data= All_data.xs('GC', level='Features', axis=1)[i].reset_index()
temp_data.columns=['ds','y']
print("\n Plot the FB-prophet model for Customer ==>",i)
pred[i] = fit_predict_model(temp_data,best_param[i])

print("\n\n Step-2 : Detect the anamalies for Customer ==>",i)
pred[i] = detect_anomalies(pred[i])
print("\n Plot the anamalies for Customer ==>",i)
plot_anomalies(pred[i])

print("Customer",i," values counts: \n",pred[i]['anomaly'].value_counts())
print("\n\n Step-3: Assign the Anomalies to the Customer",i)
All_data[i,'Anomaly'] = pred[i]['anomaly'][All_data[i].index==pred[i]['ds']].values.ravel()
All_data[i,'lower'] = pred[i]['yhat_lower'][All_data[i].index==pred[i]['ds']].values.ravel()
All_data[i,'upper'] = pred[i]['yhat_upper'][All_data[i].index==pred[i]['ds']].values.ravel()
print("\n Customer ",i,"Is Done!" )
print("==============================================================================")

In [ ]:
plot_1(All_data[i],i)

In [ ]:
plot_2(All_data[i],i)

In [ ]:
plot_3(All_data[i],i)

In [ ]:
plot_4(All_data[i]['2012-01-01':'2012-03-30'],i,3)

In [ ]:
plot_5(All_data[i]['2012-01-01':'2012-03-30'],i,3)

# ========================================

#### Third customer:

In [ ]:

i = selected_customer_id[2]
print("\n\n Identifying the Anomalies for Customer ==> ",i)
    
print("\n\n Step-1: Fit FB-prophet model with best parameters")
temp_data= All_data.xs('GC', level='Features', axis=1)[i].reset_index()
temp_data.columns=['ds','y']

print("\n Plot the FB-prophet model for Customer ==>",i)
pred[i] = fit_predict_model(temp_data,best_param[i])

print("\n\n Step-2 : Detect the anamalies for Customer ==>",i)
pred[i] = detect_anomalies(pred[i])
print("\n Plot the anamalies for Customer ==>",i)
plot_anomalies(pred[i])

print("Customer",i," values counts: \n",pred[i]['anomaly'].value_counts())
print("\n\n Step-3: Assign the Anomalies to the Customer",i)
All_data[i,'Anomaly'] = pred[i]['anomaly'][All_data[i].index==pred[i]['ds']].values.ravel()

print("\n Customer ",i,"Is Done!" )
print("==============================================================================")

In [ ]:
plot_1(All_data[i],i)

In [ ]:
plot_2(All_data[i],i)

In [ ]:
plot_3(All_data[i],i)

In [ ]:
plot_4(All_data[i]['2012-01-01':'2012-03-30'],i,3)

In [ ]:
plot_5(All_data[i]['2012-01-01':'2012-03-30'],i,3)

# ========================================

#### Forth customer:

In [ ]:

i = selected_customer_id[3]
print("\n\n Identifying the Anomalies for Customer ==> ",i)
    
print("\n\n Step-1: Fit FB-prophet model with best parameters")
temp_data= All_data.xs('GC', level='Features', axis=1)[i].reset_index()
temp_data.columns=['ds','y']

print("\n Plot the FB-prophet model for Customer ==>",i)
pred[i] = fit_predict_model(temp_data,best_param[i])

print("\n\n Step-2 : Detect the anamalies for Customer ==>",i)
pred[i] = detect_anomalies(pred[i])
print("\n Plot the anamalies for Customer ==>",i)
plot_anomalies(pred[i])

print("Customer",i," values counts: \n",pred[i]['anomaly'].value_counts())
print("\n\n Step-3: Assign the Anomalies to the Customer",i)
All_data[i,'Anomaly'] = pred[i]['anomaly'][All_data[i].index==pred[i]['ds']].values.ravel()

print("\n Customer ",i,"Is Done!" )
print("==============================================================================")

In [ ]:
plot_1(All_data[i],i)

In [ ]:
plot_2(All_data[i],i)

In [ ]:
plot_3(All_data[i],i)

In [ ]:
plot_4(All_data[i]['2012-01-01':'2012-03-30'],i,3)

In [ ]:
plot_5(All_data[i]['2012-01-01':'2012-03-30'],i,3)

# ====================================

#### Fifth customer:

In [ ]:

i = selected_customer_id[4]
print("\n\n Identifying the Anomalies for Customer ==> ",i)
    
print("\n\n Step-1: Fit FB-prophet model with best parameters")
temp_data= All_data.xs('GC', level='Features', axis=1)[i].reset_index()
temp_data.columns=['ds','y']

print("\n Plot the FB-prophet model for Customer ==>",i)
pred[i] = fit_predict_model(temp_data,best_param[i])

print("\n\n Step-2 : Detect the anamalies for Customer ==>",i)
pred[i] = detect_anomalies(pred[i])
print("\n Plot the anamalies for Customer ==>",i)
plot_anomalies(pred[i])

print("Customer",i," values counts: \n",pred[i]['anomaly'].value_counts())
print("\n\n Step-3: Assign the Anomalies to the Customer",i)
All_data[i,'Anomaly'] = pred[i]['anomaly'][All_data[i].index==pred[i]['ds']].values.ravel()

print("\n Customer ",i,"Is Done!" )
print("==============================================================================")

In [ ]:
plot_1(All_data[i],i)

In [ ]:
plot_2(All_data[i],i)

In [ ]:
plot_3(All_data[i],i)

In [ ]:
plot_4(All_data[i]['2012-01-01':'2012-03-30'],i,3)

In [ ]:
plot_5(All_data[i]['2012-01-01':'2012-03-30'],i,3)

# ======================================

#### Sixth customer:

In [ ]:

i = selected_customer_id[5]
print("\n\n Identifying the Anomalies for Customer ==> ",i)
    
print("\n\n Step-1: Fit FB-prophet model with best parameters")
temp_data= All_data.xs('GC', level='Features', axis=1)[i].reset_index()
temp_data.columns=['ds','y']

print("\n Plot the FB-prophet model for Customer ==>",i)
pred[i] = fit_predict_model(temp_data,best_param[i])

print("\n\n Step-2 : Detect the anamalies for Customer ==>",i)
pred[i] = detect_anomalies(pred[i])
print("\n Plot the anamalies for Customer ==>",i)
plot_anomalies(pred[i])

print("Customer",i," values counts: \n",pred[i]['anomaly'].value_counts())
print("\n\n Step-3: Assign the Anomalies to the Customer",i)
All_data[i,'Anomaly'] = pred[i]['anomaly'][All_data[i].index==pred[i]['ds']].values.ravel()

print("\n Customer ",i,"Is Done!" )
print("==============================================================================")

In [ ]:
plot_1(All_data[i],i)

In [ ]:
plot_2(All_data[i],i)

In [ ]:
plot_3(All_data[i],i)

In [ ]:
plot_4(All_data[i]['2012-01-01':'2012-03-30'],i,3)

In [ ]:
plot_5(All_data[i]['2012-01-01':'2012-03-30'],i,3)

# ==================================

#### Seventh customer:

In [ ]:

i = selected_customer_id[6]
print("\n\n Identifying the Anomalies for Customer ==> ",i)
    
print("\n\n Step-1: Fit FB-prophet model with best parameters")
temp_data= All_data.xs('GC', level='Features', axis=1)[i].reset_index()
temp_data.columns=['ds','y']

print("\n Plot the FB-prophet model for Customer ==>",i)
pred[i] = fit_predict_model(temp_data,best_param[i])

print("\n\n Step-2 : Detect the anamalies for Customer ==>",i)
pred[i] = detect_anomalies(pred[i])
print("\n Plot the anamalies for Customer ==>",i)
plot_anomalies(pred[i])

print("Customer",i," values counts: \n",pred[i]['anomaly'].value_counts())
print("\n\n Step-3: Assign the Anomalies to the Customer",i)
All_data[i,'Anomaly'] = pred[i]['anomaly'][All_data[i].index==pred[i]['ds']].values.ravel()

print("\n Customer ",i,"Is Done!" )
print("==============================================================================")

In [ ]:
plot_1(All_data[i],i)

In [ ]:
plot_2(All_data[i],i)

In [ ]:
plot_3(All_data[i],i)

In [ ]:
plot_4(All_data[i]['2012-01-01':'2012-03-30'],i,3)

In [ ]:
plot_5(All_data[i]['2012-01-01':'2012-03-30'],i,3)

# =====================================

#### Eighth customer:

In [ ]:

i = selected_customer_id[7]
print("\n\n Identifying the Anomalies for Customer ==> ",i)
    
print("\n\n Step-1: Fit FB-prophet model with best parameters")
temp_data= All_data.xs('GC', level='Features', axis=1)[i].reset_index()
temp_data.columns=['ds','y']

print("\n Plot the FB-prophet model for Customer ==>",i)
pred[i] = fit_predict_model(temp_data,best_param[i])

print("\n\n Step-2 : Detect the anamalies for Customer ==>",i)
pred[i] = detect_anomalies(pred[i])
print("\n Plot the anamalies for Customer ==>",i)
plot_anomalies(pred[i])

print("Customer",i," values counts: \n",pred[i]['anomaly'].value_counts())
print("\n\n Step-3: Assign the Anomalies to the Customer",i)
All_data[i,'Anomaly'] = pred[i]['anomaly'][All_data[i].index==pred[i]['ds']].values.ravel()

print("\n Customer ",i,"Is Done!" )
print("==============================================================================")

In [ ]:
plot_1(All_data[i],i)

In [ ]:
plot_2(All_data[i],i)

In [ ]:
plot_3(All_data[i],i)

In [ ]:
plot_4(All_data[i]['2012-01-01':'2012-03-30'],i,3)

In [ ]:
plot_5(All_data[i]['2012-01-01':'2012-03-30'],i,3)

# =================================

#### Ninth customer:

In [ ]:

i = selected_customer_id[8]
print("\n\n Identifying the Anomalies for Customer ==> ",i)
    
print("\n\n Step-1: Fit FB-prophet model with best parameters")
temp_data= All_data.xs('GC', level='Features', axis=1)[i].reset_index()
temp_data.columns=['ds','y']

print("\n Plot the FB-prophet model for Customer ==>",i)
pred[i] = fit_predict_model(temp_data,best_param[i])

print("\n\n Step-2 : Detect the anamalies for Customer ==>",i)
pred[i] = detect_anomalies(pred[i])
print("\n Plot the anamalies for Customer ==>",i)
plot_anomalies(pred[i])

print("Customer",i," values counts: \n",pred[i]['anomaly'].value_counts())
print("\n\n Step-3: Assign the Anomalies to the Customer",i)
All_data[i,'Anomaly'] = pred[i]['anomaly'][All_data[i].index==pred[i]['ds']].values.ravel()

print("\n Customer ",i,"Is Done!" )
print("==============================================================================")

In [ ]:
plot_1(All_data[i],i)

In [ ]:
plot_2(All_data[i],i)

In [ ]:
plot_3(All_data[i],i)

In [ ]:
plot_4(All_data[i]['2012-01-01':'2012-03-30'],i,3)

In [ ]:
plot_5(All_data[i]['2012-01-01':'2012-03-30'],i,3)

# =======================================

#### Tenth customer:

In [ ]:

i = selected_customer_id[9]
print("\n\n Identifying the Anomalies for Customer ==> ",i)
    
print("\n\n Step-1: Fit FB-prophet model with best parameters")
temp_data= All_data.xs('GC', level='Features', axis=1)[i].reset_index()
temp_data.columns=['ds','y']

print("\n Plot the FB-prophet model for Customer ==>",i)
pred[i] = fit_predict_model(temp_data,best_param[i])

print("\n\n Step-2 : Detect the anamalies for Customer ==>",i)
pred[i] = detect_anomalies(pred[i])
print("\n Plot the anamalies for Customer ==>",i)
plot_anomalies(pred[i])

print("Customer",i," values counts: \n",pred[i]['anomaly'].value_counts())
print("\n\n Step-3: Assign the Anomalies to the Customer",i)
All_data[i,'Anomaly'] = pred[i]['anomaly'][All_data[i].index==pred[i]['ds']].values.ravel()

print("\n Customer ",i,"Is Done!" )
print("==============================================================================")

In [ ]:
plot_1(All_data[i],i)

In [ ]:
plot_2(All_data[i],i)

In [ ]:
plot_3(All_data[i],i)

In [ ]:
plot_4(All_data[i]['2012-01-01':'2012-03-30'],i,3)

In [ ]:
plot_5(All_data[i]['2012-01-01':'2012-03-30'],i,3)

# =======================================

### Save the data

In [ ]:
costumer_ids = sum([ [i for j in range(All_data[i].shape[0])]for i in selected_customer_id ], [])
final_data = pd.DataFrame()
final_data['costumer'] = costumer_ids

In [ ]:
frames = [ All_data[i] for i in selected_customer_id ]
data= pd.concat(frames,axis=0, ignore_index=True)

In [ ]:
final_data= pd.concat([final_data,data],axis=1)

In [ ]:
final_data.tail()

In [ ]:
final_data.to_csv('solar_final_data.csv')

# =======================================